In [1]:
#Load library
library(tximport)
library(tidyverse)
library(biomaRt)
library(DESeq2)
library(getDEE2)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: S4Vectors

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:lubridate’:

    intersect, setdiff, union


The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’

In [2]:
#### Step2_getDEE2 OG ####
# Load data from NP
samples.og <- read.csv("Sequencing_data/NP/NP_ex_design.csv")
samples.og <- samples.og %>%
  dplyr::select(Source.Name, Comment.ENA_RUN., Comment.ENA_SAMPLE.,  Characteristics.ecotype., Characteristics.stimulus., Comment.replicate., Factor.Value.time.) %>%
  dplyr::filter(Characteristics.stimulus. == "OGs") %>%
  #dplyr::filter(Comment.replicate. == 1) %>%
  dplyr::filter(grepl(pattern = "000|030|090",Source.Name)) %>% # to extract 00min 30min 90min dataset
  dplyr::distinct() # to get unique data raw
# Choose your samples based on GEO or SRA ID
mySamples.og <- c(samples.og$Comment.ENA_RUN.)
mySamples.og.name <- c(samples.og$Source.Name)
mySamples.og.name <- unique(mySamples.og.name)
mysample.og.temp <- samples.og 
colnames(mysample.og.temp)[2] <- "SRR_accession" #rename 2nd column name for right_join
# Search metadata 
mdat <- getDEE2Metadata("athaliana")
head(mdat)
# Since mySamples have SRR_accession info, we need to change them to SRP_accession vector
mdat.mySamples.og <- mdat %>% dplyr::filter(SRR_accession %in% mySamples.og) %>% # make a dataframe which have only our samples
  right_join(mysample.og.temp, by = "SRR_accession") %>% # add information from OG_sample info to arrange
  dplyr::filter(!is.na(QC_summary)) %>% # remove NA because these dataset were not include in mdat
  dplyr::arrange(Factor.Value.time.) # arrange by treatment time 
mySamples.og.SRR <- mdat.mySamples.og$SRR_accession # get SRR information
SRRvec.og <- as.vector(mySamples.og.SRR) # format change as vector with order

#Extract expression data from DEE2
expression.og <- getDEE2("athaliana", SRRvec.og, 
                         metadata = mdat, 
                         counts = "Tx2Gene", 
                         legacy = TRUE)
#species, SRRvec, metadata df, counts method
## counts method : "GeneCounts" (STAR), "TxCounts" (Kallisto), "Tx2Gene" (Kallisto count aggregated by sum)
## legacy: TRUE -> return list data, default -> return SummarizedExperiment

,SRR_accession,QC_summary,SRX_accession,SRS_accession,SRP_accession,Experiment_title,GEO_series
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,DRR000618,"FAIL(3,4)",DRX000325,DRS000288,DRP000209,Arabidopsis Transcriptome Multiplex-1_No1,
2,DRR000619,"FAIL(3,4,6)",DRX000326,DRS000289,DRP000209,Arabidopsis Transcriptome Multiplex-1_No3,
3,DRR000620,"FAIL(3,4,6)",DRX000327,DRS000290,DRP000209,Arabidopsis Transcriptome Multiplex-1_No4,
4,DRR000621,"FAIL(3,4,6)",DRX000328,DRS000291,DRP000209,Arabidopsis Transcriptome Multiplex-1_No5,
5,DRR008476,PASS,DRX007662,DRS007600,DRP001015,Arabidopsis WT-Col mRNA_seq,
6,DRR008477,PASS,DRX007663,DRS007601,DRP001015,Arabidopsis ibm1-4 mRNA_seq,


For more information about DEE2 QC metrics, visit
    https://github.com/markziemann/dee2/blob/master/qc/qc_metrics.md



In [3]:
#### Step2_DESeq OG ####
# Tximport OGs data to DEseq2
# Make a matrix of OG treatment TxCounts
#input.og <- expression.og$TxCounts
#input.og <- round(input.og) #round counts to make integer
input.og <- expression.og$GeneCounts

# Make sample information
sample.og <- mdat.mySamples.og %>%
  dplyr::select("SRR_accession", "Source.Name", "Comment.replicate.", "Factor.Value.time.")
sample.og$condition <- factor(rep(c("A", "B", "C"),each=4))
sample.og <- sample.og[order(sample.og$SRR_accession),]
rownames(sample.og) <- sample.og$SRR_accession #df is ordered by SRR_accession

# DEseq using Matrix import with og matrix file: input.og
ddsog <- DESeqDataSetFromMatrix(countData=input.og, colData = sample.og, design = ~ condition)
ddsog.dds <- DESeq(ddsog)
colnames(ddsog.dds) <- sample.og$Source.Name

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



In [4]:
# Get results comparing with og 0min treatment
res.30min.og <- results(ddsog.dds, contrast=c('condition', 'B', 'A'))
res.90min.og <- results(ddsog.dds, contrast=c('condition', 'C', 'A'))
res.30min.og.df <- as.data.frame(res.30min.og)
res.90min.og.df <- as.data.frame(res.90min.og)
res.30min.og.full <- merge(res.30min.og.df, as.data.frame(counts(ddsog.dds, normalized=TRUE)), by="row.names", sort=FALSE)
res.90min.og.full <- merge(res.90min.og.df, as.data.frame(counts(ddsog.dds, normalized=TRUE)), by="row.names", sort=FALSE)
names(res.30min.og.full)[1] <- "Gene"
names(res.90min.og.full)[1] <- "Gene"

# Rearrange OG file by treatment time
res.30min.og.full <- res.30min.og.full %>%
  dplyr::select("Gene", "baseMean", "log2FoldChange", "lfcSE", "stat", "pvalue", "padj", 
                "R1_Col_OGs_000", "R2_Col_OGs_000", "R3_Col_OGs_000", "R4_Col_OGs_000",
                "R1_Col_OGs_030", "R2_Col_OGs_030", "R3_Col_OGs_030", "R4_Col_OGs_030",
                "R1_Col_OGs_090", "R2_Col_OGs_090", "R3_Col_OGs_090", "R4_Col_OGs_090")
res.90min.og.full <- res.90min.og.full %>%
  dplyr::select("Gene", "baseMean", "log2FoldChange", "lfcSE", "stat", "pvalue", "padj", 
                "R1_Col_OGs_000", "R2_Col_OGs_000", "R3_Col_OGs_000", "R4_Col_OGs_000",
                "R1_Col_OGs_030", "R2_Col_OGs_030", "R3_Col_OGs_030", "R4_Col_OGs_030",
                "R1_Col_OGs_090", "R2_Col_OGs_090", "R3_Col_OGs_090", "R4_Col_OGs_090")

# Generate og full DESeq results file and export
res.30min.og.full <- res.30min.og.full %>%
  dplyr::arrange(Gene)
res.90min.og.full <- res.90min.og.full %>%
  dplyr::arrange(Gene)
#write_csv(res.30min.og.full, "og_30min.csv")
#write_csv(res.90min.og.full, "og_90min.csv")
write_csv(res.30min.og.full, "01.DESeq_results/og_30min_gene.csv")
print("01.DESeq_results/og_30min_gene.csv")
write_csv(res.90min.og.full, "01.DESeq_results/og_90min_gene.csv")
print("01.DESeq_results/og_90min_gene.csv")

[1] "01.DESeq_results/og_30min_gene.csv"
[1] "01.DESeq_results/og_90min_gene.csv"
